### Display a mosaic of the LSST camera

In [1]:
import os
import numpy as np
#import lsst.daf.persistence as dafPersist
from lsst.daf.butler import Butler
import lsst.afw.cameraGeom.utils as cameraGeomUtils
import lsst.afw.display as afwDisplay
import matplotlib.pyplot as plt   # Not needed for image displays, but used for analysis

%matplotlib ipympl
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = (10, 7)

In [2]:
afwDisplay.setDefaultBackend("matplotlib")

In [6]:
dataRoot = "/lsstdata/offline/teststand/BOT/gen3repo"

butler = Butler(dataRoot, collections=['LSSTCam/raw/all', 'LSSTCam/calib'])
dataId0 = dict(instrument='LSSTCam')
run = str(12526)

dataset = next(iter(butler.registry.queryDatasets('raw', collections='LSSTCam/raw/all', dataId=dataId0,
                                                  where=f"exposure.science_program = '12526'")))
exposure = dataset.dataId["exposure"]  # I need an exposure to get a camera
del dataset

camera = butler.get('camera', dataId0, exposure=exposure)

dataId = dataId0.copy()
dataId["exposure.science_program"] = run
dataId["detector"] = camera['R22_S11'].getId()

LookupError: Dataset camera with data ID {instrument: 'LSSTCam', exposure: 3020082500026, ...} could not be found in collections [LSSTCam/raw/all, LSSTCam/calib].

It'll be easier to see if we correct for the gains.

To do this, we'll define a custom callback:

In [7]:
def myCallback(im, ccd, imageSource):
    """Assemble the CCD image, subtracting the overscan correcting for the gain"""

    return cameraGeomUtils.rawCallback(im, ccd, imageSource,
                                       subtractBias=True, correctGain=False)

In [8]:
%%time
seqNum = 32

exposureRec = next(iter(butler.registry.queryDimensionRecords("exposure", dataId=dataId0,
                                                              where=f'exposure.science_program={dataId["exposure.science_program"]} AND '
                                                                    f'exposure.seq_num = {seqNum}')))
#dayObs = exposureRec.day_obs
exposure = exposureRec.id

disp = afwDisplay.Display(1, reopenPlot=True)
disp.scale('asinh', 'zscale', Q=2)
disp.scale('linear', 'minmax')
disp.scale('asinh', 5, 7000, Q=2)

disp.setImageColormap('viridis' if True else 'gray')

#
# Set detectorNameList to only display a subset of the detectors
#
if True:   # just a single raft
    detectorNameList = [det.getName() for det in camera if "R00" in det.getName()]
elif False:
    detectorNameList = ['R22_S11']
else:
    detectorNameList = [camera[det].getName() for det in butler.queryMetadata('raw', ['detector'], run=dataId['run'], seqNum=seqNum)]

detectorNameList = None

dataType = "raw"
mos = cameraGeomUtils.showCamera(camera,
                                 cameraGeomUtils.ButlerImage(butler, dataType, instrument='LSSTCam', exposure=exposure,
                                                             #run=dataId["run"], seqNum=seqNum,
                                                             callback=myCallback, verbose=True),
                                 binSize=16, detectorNameList=detectorNameList, display=disp, overlay=False,
                                 title=f'{dataId["exposure.science_program"]} {seqNum}')

NameError: name 'dataId' is not defined

In [ ]:
#plt.savefig("BOT-Flammarion.png")

In [5]:
disp = afwDisplay.Display(1, reopenPlot=True)
disp.setImageColormap('viridis' if False else 'gray')

disp.scale('asinh', 'zscale', Q=2)

disp.mtv(mos)
plt.title(f'{dataId["run"]} {seqNum}');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'mos' is not defined

Just assemble the corner rafts, and use matplotlib to make the mosaic

In [7]:
fig = plt.figure()
fig.subplots_adjust(hspace=0.2, wspace=0.0)

disp = afwDisplay.Display(fig)
disp.scale('asinh', 'zscale', Q=2)
disp.scale('linear', 'minmax')
disp.scale('asinh', 5, 7000, Q=2)

disp.setImageColormap('viridis' if True else 'gray')
dataType = "raw"
for i, raftName in enumerate(['R40', 'R44', 'R00', 'R04'], 1):
    ax = fig.add_subplot(2, 2, i)
    
    detectorNameList = [det.getName() for det in camera if f"{raftName}" in det.getName()]

    mos = cameraGeomUtils.showCamera(camera,
                                     cameraGeomUtils.ButlerImage(butler, dataType, instrument='LSSTCam', exposure=exposure,
                                                                 #run=dataId["run"], seqNum=seqNum,
                                                                 callback=myCallback, verbose=True),
                                     binSize=16, detectorNameList=detectorNameList, display=disp, overlay=False,
                                     title=raftName)
    disp.show_colorbar(False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

MC_C_20200825_000032 R40_SG0: No FILTER key found but FILTER2="ND_OD1.0" (removed)
MC_C_20200825_000032 R40_SG1: No FILTER key found but FILTER2="ND_OD1.0" (removed)
MC_C_20200825_000032 R40_SW0: No FILTER key found but FILTER2="ND_OD1.0" (removed)
MC_C_20200825_000032 R40_SW1: No FILTER key found but FILTER2="ND_OD1.0" (removed)
MC_C_20200825_000032 R44_SG0: No FILTER key found but FILTER2="ND_OD1.0" (removed)
MC_C_20200825_000032 R44_SG1: No FILTER key found but FILTER2="ND_OD1.0" (removed)
MC_C_20200825_000032 R44_SW0: No FILTER key found but FILTER2="ND_OD1.0" (removed)
MC_C_20200825_000032 R44_SW1: No FILTER key found but FILTER2="ND_OD1.0" (removed)
MC_C_20200825_000032 R00_SG0: No FILTER key found but FILTER2="ND_OD1.0" (removed)
MC_C_20200825_000032 R00_SG1: No FILTER key found but FILTER2="ND_OD1.0" (removed)
MC_C_20200825_000032 R00_SW0: No FILTER key found but FILTER2="ND_OD1.0" (removed)
MC_C_20200825_000032 R00_SW1: No FILTER key found but FILTER2="ND_OD1.0" (removed)
MC_C